In [1]:
import numpy as np
import cv2
import pathlib
import os
from scipy.linalg import null_space
import sympy

class Camera:
    
    def __init__(self, val):
        self.img_num = val # カメラ番号（int;コンストラクタ）
        
        f = 8000/3
        cx = 1920/2
        cy = 1080/2
        A = np.zeros((3,3))
        A[0,0] = f
        A[0,2] = cx
        A[1,1] = f
        A[1,2] = cy
        A[2,2] = 1
        
        self.A = A # 内部パラメータ(ndarray)

    def img_load(self):
        folder_path = "image"
        file_path = os.path.join(folder_path, str(self.img_num) + ".png")
        img = cv2.imread(file_path, 1)# BGRで読み込み
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        self.img = img # 画像(ndarray)
        
    def contour_extraction(self):
        
        color_arr = np.array([[255,0,0],[0,255,0],[0,0,255],
                             [255,255,0],[255,0,255],[0,255,255],
                             [127,127,127],[127,0,127],[0,127,127]],dtype = np.int16)
        masks = np.ones((self.img.shape[0], self.img.shape[1], 9), dtype=np.uint8)
        
        for i, color in enumerate(color_arr):
            lower = np.clip(color, 0, 255)
            upper = np.clip(color, 0, 255)
            img_mask = cv2.inRange(self.img, lower, upper)
            masks[:,:,i] = img_mask
        
        #self.masks = masks # 色ごとのマスク(nd.array)
        
        contour_list = []

        # 色ごとに輪郭（閉曲線）を抽出
        for i in range(masks.shape[2]):
            contours, hierarchy = cv2.findContours(masks[:,:,i],cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            contour_list.append(contours)
        self.contour_list = contour_list # 輪郭のリスト(list,ndarray)

        #self.frag_list = contours2fragments(self.contour_list) # フラグメントのリスト(list,ndarray)

    def para_load(self):

        folder_path = pathlib.Path("view_mat")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.Rt = np.loadtxt(file_path, delimiter="\t")
        self.P = np.dot(self.A, self.Rt[0:3,0:4])
        
        folder_path = pathlib.Path("cam_pos_rot")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.cam_world_cood = np.loadtxt(file_path, delimiter=',')[0]

In [2]:
#曲線分割
def split_list(contour_length, max_frag_len=100, min_frag_len=40, min_overrap=10):
    
    # 輪郭のフラグメントの位置を指定(最小40 pixl)
    if contour_length > max_frag_len:
        pass
    
    elif contour_length < 40:
        return None
    
    elif contour_length == 40:
        return [[0,39]]
    
    else:
        max_frag_len = contour_length
    
    step0 = np.random.randint(min_frag_len, max_frag_len) # 一つ目のフラグメントの長さ（40から100）
    sep_list = [[0,step0]]
    back = np.random.randint(min_overrap, step0-1) # フラグメントを重ねるために戻す分を決める（最小10 pixl）
    next_start = step0 - back
    
    while True:
        
        # 戻った分(back)より進む
        if back+1 > min_frag_len:
            step = np.random.randint(back+1, max_frag_len)
        else:
            step = np.random.randint(min_frag_len, max_frag_len)

        full_length = next_start + step
        sept = [next_start, full_length]
        sep_list.append(sept)
        back = np.random.randint(10, step-1)
        next_start = full_length - back

        # 終了判定
        if full_length > contour_length:
            break
    
    # 超過した分戻す（長さはそのまま）
    difference = sep_list[-1][1] - (contour_length-1)
    sep_list[-1][0] -= difference
    sep_list[-1][1] -= difference
    
    return sep_list


def contours_split(contour):
    
    #contour.shape == (N, 2)
    contour_length = contour.shape[0]
    sp_list = split_list(contour_length)
    
    if sp_list == None:
        return None
    
    frag_list = []
    # 位置のリスト通りにスライス
    for sp in sp_list:
        #print(sp)
        frag_list.append(contour[sp[0]:sp[1],:])

    return frag_list


def all_fraged(contours_list):
    
    # 輪郭のリストからフラグメントのリストを得る
    frags_list = []

    #for i in contours_list:
    #temp_list = []
    frags = []
    for j in contours_list:
        temp_frags = contours_split(j.squeeze())
            
        if temp_frags != None:
            frags += temp_frags

    #if frags != []:
    #    frags_list.append(frags)
    
    return frags

def frag_list_fraged(frags_list):# frags_list[色][輪郭][sep][座標]
    img_frag_list = []
    for frag in frags_list:
        color_frag = all_fraged(frag)
        img_frag_list.append(color_frag)
    return img_frag_list

In [3]:
# camera対応
def dim3_distance(vec1, vec2):
    return sum((vec1-vec2)**2)

def camera_correspondence(cam_list):
    vec_list = []
    for i,cam in enumerate(cam_list):
        cam_list[i].para_load()
        vec_list.append(cam_list[i].cam_world_cood)

    pair_list = []
    for i, vec1 in enumerate(vec_list):
        for j, vec2 in enumerate(vec_list):
            if i == j or i > j :
                continue
            elif dim3_distance(vec1,vec2) < 2:
                pair_list.append((i,j))
    
    return pair_list

In [4]:
# epipole取得

def SS_mat(vec3):
    vec3 = np.squeeze(vec3)
    SS_mat = np.zeros((3, 3))
    SS_mat[0,1] = -vec3[2]
    SS_mat[0,2] = vec3[1]
    SS_mat[1,0] = vec3[2]
    SS_mat[1,2] = -vec3[0]
    SS_mat[2,0] = -vec3[1]
    SS_mat[2,1] = vec3[0]
    return SS_mat

def FF_mat(A1, A2, Rt1, Rt2):
    P1 = np.dot(A1, Rt1[0:3,0:4])
    P2 = np.dot(A2, Rt2[0:3,0:4])
    cam_pos = -np.dot(Rt1[0:3,0:3].T, Rt1[0:3,3])
    cam_pos = np.array([cam_pos[0], cam_pos[1], cam_pos[2], 1])
    epipole2 = np.dot(P2, cam_pos)
    cam_pos = -np.dot(Rt2[0:3,0:3].T, Rt2[0:3,3])
    cam_pos = np.array([cam_pos[0], cam_pos[1], cam_pos[2], 1])
    epipole1 = np.dot(P1, cam_pos)
    return epipole1, epipole2, np.dot(SS_mat(epipole2), np.dot(P2, np.linalg.pinv(P1)))

def contour_disassembly(contour_list):
    con_dis = []
    for i in range(len(contour_list)):
        if contour_list[i] == []:
            continue
        which_dis = np.concatenate(contour_list[i])
        con_dis.append(which_dis)
        
    return np.concatenate(con_dis)

In [5]:
# エピポーラ関係
def epilines_para(frags, F):
    
    lines_list = []
    for color in frags:
        temp_color_list = []
        for frag in color:
            frag_lines = cv2.computeCorrespondEpilines(frag.reshape(-1,1,2), 1,F) # ndarray(フラグメントの座標数,1,3)
            temp_color_list.append(frag_lines)

        lines_list.append(temp_color_list)
    
    return lines_list


def para2line(parameter):
    
    #一つのパラメータが渡された時を想定
    line_coode = np.zeros((1920,2), dtype=np.int64)
    para = np.squeeze(parameter)# 3次ベクトル
    for x in range(1920):
        y = int((-para[0]*x - para[2])/para[1])
        line_coode[x,0] = x
        line_coode[x,1] = y
    
    return line_coode


def epiline_cal(frag_paras):
    #全ての色に対するエピポーラ線の帯の計算
    #lines[色][フラグメント][線][座標]
    lines = []
    for color in frag_paras:
        temp_color = []
        
        for frag in color:
            temp_line = []

            for point in frag:
                line = para2line(point)
                temp_line.append(line)

            temp_color.append(temp_line)
        lines.append(temp_color)
        
    return lines


def frags_vs_line(img2_frags, frag_epiline):
    
    # frag_epiline shape(1920, 2)
    surport = np.zeros(len(img2_frags))
    for i in frag_epiline:
        for j, frag in enumerate(img2_frags):
            if i in frag:
                surport[j] += 1
    
    return surport


def pair_frag_idx(img2_frags, frag_epilines):
    surport = np.zeros(len(img2_frags))
    for epi in frag_epilines:
        surport += frags_vs_line(img2_frags, epi)
        
    return surport, np.argmax(surport)


In [6]:
def normalization(vec3):
    return vec3[0]/vec3[2], vec3[1]/vec3[2]

In [7]:
# 再構築
def nom_F(F):
    return (1/sum(sum(F**2))**(1/2))*F

def cover_mat(x1, y1, x2, y2):
    return np.array([[x1**2+x2**2, x2*y2, x2, x1*y1, 0, 0, x1, 0, 0],
                    [x2*y2, x1**2+y2**2, y2, 0, x1*y1, 0, 0, x1, 0],
                    [x2, y2, 1, 0, 0, 0, 0, 0, 0],
                    [x1*y1, 0, 0, y1**2+x2**2, x2*y2, x2, y1, 0, 0],
                    [0, x1*y1, 0, x2*y2, y1**2+y2**2, y2, 0, y1, 0],
                    [0, 0, 0, x2, y2, 1, 0, 0, 0],
                    [x1, 0, 0, y1, 0, 0, 1, 0, 0],
                    [0, x1, 0, 0, y1, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0]])

def min_dist(F, pt1, pt2):# pt1が画像2上の点，pt2が画像1上の点
    S0 = 10**10
    x1_ori = pt1[0]
    y1_ori = pt1[1]
    x2_ori = pt2[0]
    y2_ori = pt2[1]
    
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    
    x1_tilda = 0
    y1_tilda = 0
    x2_tilda = 0
    y2_tilda = 0
    thita = nom_F(F).flatten()

    while True:
        V_eps = cover_mat(x1, y1, x2, y2)
        eps_ast = np.array([x1*x2 + x2*x1_tilda + x2*x2_tilda,
                           x1*y2 + y2*x1_tilda + x2*y2_tilda,
                           x1 + x1_tilda,
                           y1*x2 + x2*y1_tilda + y1 * x2_tilda,
                           y1* y2 + y2*y1_tilda + y1*y2_tilda,
                           y1 + y1_tilda,
                           x2 + x2_tilda,
                           y2 + y2_tilda,
                           1])
        
        x1_y1_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[1], thita[2]], [thita[3], thita[4], thita[5]]]), np.array([x2, y2, 1])) / np.dot(thita, np.dot(V_eps, thita))
        x2_y2_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[3], thita[6]], [thita[1], thita[4], thita[7]]]), np.array([x1, y1, 1])) / np.dot(thita, np.dot(V_eps, thita))
        
        x1_tilda = x1_y1_tilda[0]
        y1_tilda = x1_y1_tilda[1]
        x2_tilda = x2_y2_tilda[0]
        y2_tilda = x2_y2_tilda[1]
        
        x1 = x1_ori - x1_tilda
        y1 = y1_ori - y1_tilda
        x2 = x2_ori - x2_tilda
        y2 = y2_ori - y2_tilda
        
        S = x1_tilda**2 + y1_tilda**2 + x2_tilda**2 + y2_tilda**2
        
        if S0 - S == 0:
            break
        else:
            S0 = S

    return x1, y1, x2, y2

def Ps(P, pt):
    a = P[0,0] - pt[0]*P[2,0]
    b = P[0,1] - pt[0]*P[2,1]
    c = P[0,2] - pt[0]*P[2,2]
    d = P[0,3]
    e = pt[0]*P[2,3]
    f = P[1,0] - pt[1]*P[2,0]
    g = P[1,1] - pt[1]*P[2,1]
    h = P[1,2] - pt[1]*P[2,2]
    i = P[1,3]
    j = pt[1]*P[2,3]
    return a, b, c, d, e, f, g, h, i, j

def tri(P1, P2, pt1, pt2):
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    z = sympy.Symbol('z')
    a1, b1, c1, d1, e1, f1, g1, h1, i1, j1 = Ps(P1, pt1)
    a2, b2, c2, d2, e2, f2, g2, h2, i2, j2 = Ps(P2, pt2)
    
    expr1 = a1*x + b1*y + c1*z + d1 - e1
    expr2 = f1*x + g1*y + h1*z + i1 - j1
    expr3 = a2*x + b2*y + c2*z + d2 - e2
    expr4 = f2*x + g2*y + h2*z + i2 - j2
    print(expr1.subs(sympy.solve([expr2, expr3, expr4])))
    
    return sympy.solve([expr2, expr3, expr4])

In [8]:
import matplotlib.pyplot as plt

In [9]:
cam_list = [Camera(i) for i in range(24)]
for i in range(len(cam_list)):
    cam_list[i].img_load()
    cam_list[i].contour_extraction()
    cam_list[i].para_load()
#cam_pairs = camera_correspondence(cam_list)

In [10]:
# エピポール取得
cam_pairs = camera_correspondence(cam_list)

epipole_dict = {i:[] for i in range(len(cam_list))}
cam_pairs_F = {}
for i in cam_pairs:
    epipole1, epipole2, F = FF_mat(cam_list[i[0]].A, cam_list[i[1]].A, cam_list[i[0]].Rt, cam_list[i[1]].Rt)
    epipole_dict[i[0]].append(normalization(epipole1))
    epipole_dict[i[1]].append(normalization(epipole2))
    cam_pairs_F[i] = F

In [11]:
#エピポールと輪郭上の点を結んだ直線と，x軸のなす角
def gene(angles):
    # 正規化
    B = list(map(lambda y:y-min(angles),angles))
    return list(map(lambda y: (y-min(B))/(max(B)-min(B)), B))

def epipole_angle(img_num, epipole_dict):
    cam = cam_list[img_num]
    cam.img_load()
    cam.contour_extraction()
    angle_list = []
    
    for epi in epipole_dict[img_num]:
        epi_angle_list = []
        epi_x = epi[0]
        epi_y = epi[1]
        for color in cam.contour_list:
            color_angle_list = []
            for contour in color:
                contour_angle_list= []
                for cood in contour:
                    x = cood[0][0]
                    y = cood[0][1]
                    tilt = (y-epi_y)/(x-epi_x)
                    angle = np.arctan(tilt)
                    contour_angle_list.append(angle)
                contour_angle_list = gene(contour_angle_list)
                color_angle_list.append(contour_angle_list)
            epi_angle_list.append(color_angle_list)
        angle_list.append(epi_angle_list)
    return angle_list
    
def expand(idx_l, list_length):
    del_list = []
    for i in idx_l:
        if np.isnan(i):
            continue
        if i-2 < 0:
            del_list.append(list_length + i-2)
        else:
            del_list.append(i-2)
        
        if i-1 < 0:
            del_list.append(list_length + i-1)
        else:
            del_list.append(i-1)
        
        del_list.append(i)
        
        if i+1 > list_length-1:
            del_list.append(i+1-list_length)
        else:
            del_list.append(i+1)
        if i+2 > list_length-1:
            del_list.append(i+2-list_length)
        else:
            del_list.append(i+2)
    return sorted(list(set(del_list)))

def differential(angles):
    # エピポーラ線に平行な接線をもつ点(前後方微分の正負を比べたほうが良い)
    del_idx = []
    for i in range(len(angles)):
        if np.isnan(angles[i]):
            continue
        if i == len(angles)-1:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[0]-angles[i]): #or abs(angles[0]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
        else:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[i+1]-angles[i]):# or abs(angles[i+1]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
    #del_idx = expand(del_idx, len(angles))
    
    return del_idx

def marge_del(epi_del_list):
    im_del_list = []
    for i, col in enumerate(epi_del_list[0]):
        color_list = []
        for j, con in enumerate(col):
            color_list.append(list(set(epi_del_list[0][i][j] + epi_del_list[1][i][j])))
        im_del_list.append(color_list)
    return im_del_list

def all_D(angles_list):
    # 画像1枚に対して削除リストを作成
    all_del_list = []
    for epi in angles_list:
        epi_del_list = []
        for color in epi:
            color_del_list = []
            for contour in color:
                #if len(contour)<40:
                #    continue
                del_idx = differential(contour)
                color_del_list.append(del_idx)
            epi_del_list.append(color_del_list)
        all_del_list.append(epi_del_list)
    all_del_list = marge_del(all_del_list)
    return all_del_list

In [12]:
# エピポーラ線との接線で分割
def separate(contour, del_idx):
    # 一つの輪郭に対し削除リストから削除
    start = 0
    newArray = []
    for d in del_idx:
        if contour[start:d] != []:
            if contour[start:d].size != 0:
                newArray.append(contour[start:d])
        start = d+1

    if contour[start:].size != 0:
        newArray.append(contour[start:])
    return newArray

def all_sep(con_list, del_list):
    n_list = []
    for col, del_col in zip(con_list, del_list):
        n_col_list = []
        for con, del_con in zip(col, del_col) :
            n_con = separate(con, del_con)
            for frag in n_con:
                n_col_list.append(frag)
        n_list.append(n_col_list)
    return n_list

In [13]:
# フラグメントのリストを作る
for i in range(len(cam_list)):
    im_del_list = all_D(epipole_angle(i, epipole_dict))# im_del_list[color][contour][del_idx]
    newCon = all_sep(cam_list[i].contour_list, im_del_list)# newCon[color][fragment][coordination]
    cam_list[i].frag_list = frag_list_fraged(newCon)

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


In [14]:
# 衝突判定
def para2cood_S(para_list):
    return np.array([[0, -c/b] for a,b,c in para_list], dtype=np.int64)

def para2cood_F(para_list):
    return np.array([[1920, -(1920*a+c)/b] for a,b,c in para_list], dtype=np.int64)

def all_pa2co(para_list):
    epi_cood_S = []
    epi_cood_F = []
    for color in para_list:
        color_list_S = []
        color_list_F = []
        for frag in color:
            S_cood = para2cood_S(frag.squeeze())
            F_cood = para2cood_F(frag.squeeze())
            color_list_S.append(S_cood)
            color_list_F.append(F_cood)
        epi_cood_S.append(color_list_S)
        epi_cood_F.append(color_list_F)
    return epi_cood_S, epi_cood_F # epi_cood[color][frag]

def get_frag_cood(frag_list):
    cood_S = []
    cood_F = []
    for color in frag_list:
        col_S = np.array([frag[0] for frag in color])
        col_F = np.array([frag[-1] for frag in color])
        cood_S.append(col_S)
        cood_F.append(col_F)
    return cood_S, cood_F # cood_S[color][frag]

def coll_t1_t2(epi_cood_S, epi_cood_F, cood_S, cood_F):
    epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_S])
    epi_cood_F_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_F])
    v = cood_S - epi_cood_S_bro
    v2 = cood_F - cood_S
    v1 = epi_cood_F_bro - epi_cood_S_bro
    t1 = np.cross(v, v2)/np.cross(v1, v2)
    t2 = np.cross(v, v1)/np.cross(v1, v2)
    return t1, t2

def coll_det(t1, t2):
    t1_t = np.array((t1 <= 1) & (t1 > 0),dtype=np.int16)
    t2_t = np.array((t2 <= 1) & (t2 > 0),dtype=np.int16)
    count_c = np.array(t1_t + t2_t == 2, dtype=np.int64)
    surport_idx = np.argmax(np.sum(count_c,axis=0))
    return surport_idx

def make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F):
    img_list=[]
    for epi_S_col, epi_F_col, S_col, F_col in zip(epi_cood_S, epi_cood_F, cood_S, cood_F):
        color_list = []
        if len(epi_S_col) == 0 or len(S_col) == 0:
            img_list.append(color_list)
            continue
        for epi_S_frag, epi_F_frag in zip(epi_S_col, epi_F_col):
            t1, t2 = coll_t1_t2(epi_S_frag, epi_F_frag, S_col, F_col)
            surport_idx = coll_det(t1, t2)
            color_list.append(surport_idx)
        img_list.append(color_list)
    return img_list

In [15]:
pair_list = {}
for pair in cam_pairs_F:
    F = cam_pairs_F[pair]
    frags_para12 = epilines_para(cam_list[pair[0]].frag_list, F) # frags_para[色][frag]
    frags_para21 = epilines_para(cam_list[pair[1]].frag_list, F.T)
    
    cood_S, cood_F = get_frag_cood(cam_list[pair[1]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para12)
    img_list1 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    cood_S, cood_F = get_frag_cood(cam_list[pair[0]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para21)
    img_list2 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    pair_list[pair[0],pair[1]] = img_list1
    pair_list[pair[1],pair[0]] = img_list2

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in less_equal
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater


In [17]:
len(cam_list[0].frag_list[0])

361

In [175]:
for i in pair_list:
    print(i)

(0, 2)
(2, 0)
(0, 21)
(21, 0)
(1, 6)
(6, 1)
(1, 12)
(12, 1)
(2, 14)
(14, 2)
(3, 6)
(6, 3)
(3, 8)
(8, 3)
(4, 15)
(15, 4)
(4, 22)
(22, 4)
(5, 8)
(8, 5)
(5, 22)
(22, 5)
(7, 17)
(17, 7)
(7, 20)
(20, 7)
(9, 13)
(13, 9)
(9, 18)
(18, 9)
(10, 14)
(14, 10)
(10, 17)
(17, 10)
(11, 19)
(19, 11)
(11, 23)
(23, 11)
(12, 18)
(18, 12)
(13, 15)
(15, 13)
(16, 19)
(19, 16)
(16, 20)
(20, 16)
(21, 23)
(23, 21)


In [179]:
it = 0
for pair in pair_list:
    if it % 2 == 0:
        F = cam_pairs_F[pair]
        frags_para = epilines_para(cam_list[pair[0]].frag_list, F) # frags_para[色][frag]
    elif it % 2 == 1:
        F = cam_pairs_F[pair[1], pair[0]]
        frags_para = epilines_para(cam_list[pair[0]].frag_list, F.T) # frags_para[色][frag]
        
    epi_cood_S, epi_cood_F = all_pa2co(frags_para)
    for pair_col, col_part, epi_S_col, epi_F_col in zip(pair_list[pair], cam_list[pair[1]].frag_list, epi_cood_S, epi_cood_F):
        for pair_frag, epi_S_frag, epi_F_frag in zip(pair_col, epi_S_col, epi_F_col):
            pts = col_part[pair_frag] # 対応するフラグメント
            v1 = epi_F_frag - epi_S_frag
            v1_n = (v1[:,0]**2+v1[:,1]**2)**(1/2)
            v1_n = np.stack([v1_n, v1_n], axis=1)
            v1 = v1/v1_n
            v1_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in v1])
            epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in epi_S_frag])
            v2 = pts - epi_cood_S_bro
            v2_n = (v2[:,:,0]**2+v2[:,:,1]**2)**(1/2)
            v2_n = np.stack([v2_n, v2_n], axis=2)
            v2 = v2/v2_n
            con_det = np.cross(v1_bro, v2)
            print(np.where((np.abs(con_det) <= 0.001) & (con_det == np.min(np.abs(con_det)))))
            break
        break
    break
    it += 1


(array([29, 30], dtype=int64), array([62, 61], dtype=int64))


In [181]:
np.min(con_det,axis=0).size

87

In [154]:
v1 = (v1[:,0]**2 + v1[:,1]**2)**(1/2)

SyntaxError: invalid syntax (<ipython-input-154-c9da6c19b754>, line 1)

In [161]:
np.stack([v1, v1], axis=1).shape

(69, 2)

In [167]:
np.abs(con_det[19])

array([0.03162107, 0.03172193, 0.03182351, 0.0319258 , 0.03202881,
       0.03213256, 0.03223705, 0.03234229, 0.03244828, 0.03255503,
       0.03266256, 0.03277087, 0.03287996, 0.03298986, 0.03310056,
       0.03321207, 0.03332442, 0.03332442, 0.03343759, 0.03343759,
       0.03355161, 0.03366648, 0.03378221, 0.03389882, 0.0340163 ,
       0.0340163 , 0.0340163 , 0.03413468, 0.03413468, 0.03425396,
       0.03425396, 0.03425396, 0.03437415, 0.03449527, 0.03461731,
       0.03474031, 0.03486425, 0.03498917, 0.03511506, 0.03524194,
       0.03536983, 0.03549872, 0.03562864, 0.0357596 , 0.03589161,
       0.03602468, 0.03615882, 0.03629406, 0.03643039, 0.03643039,
       0.03656784, 0.03670642, 0.03684615, 0.03698703, 0.03712908,
       0.03727232, 0.03741676, 0.03756241, 0.0377093 , 0.03785743,
       0.03800683, 0.03815751, 0.03830948, 0.03846277, 0.03846277,
       0.03861739, 0.03877335, 0.03893068, 0.03908939, 0.03924951,
       0.03941104, 0.03957402, 0.03957402, 0.03973845, 0.03990

In [79]:
print(len(v1),len(epi_F_frag))

69 69


In [34]:
con_det.shape

(83, 91)

In [75]:
len(cam_list[0].frag_list[0][0])

69

In [74]:
len(cam_list[2].frag_list[0][372])

87

In [90]:
for i,j in enumerate(cam_list[2].frag_list[0][372]):
    print(i,j)

0 [895 817]
1 [894 816]
2 [894 815]
3 [893 814]
4 [892 813]
5 [892 812]
6 [892 811]
7 [891 810]
8 [891 809]
9 [890 808]
10 [889 807]
11 [889 806]
12 [888 805]
13 [887 804]
14 [886 803]
15 [886 802]
16 [886 801]
17 [885 801]
18 [884 800]
19 [883 800]
20 [882 799]
21 [882 798]
22 [881 797]
23 [880 796]
24 [879 795]
25 [878 795]
26 [877 795]
27 [876 794]
28 [875 794]
29 [874 793]
30 [873 793]
31 [872 793]
32 [871 792]
33 [871 791]
34 [870 790]
35 [870 789]
36 [870 788]
37 [870 787]
38 [871 786]
39 [871 785]
40 [872 784]
41 [873 783]
42 [874 782]
43 [875 781]
44 [876 780]
45 [877 779]
46 [878 778]
47 [878 777]
48 [879 776]
49 [880 776]
50 [881 775]
51 [882 774]
52 [883 773]
53 [883 772]
54 [884 771]
55 [885 770]
56 [886 769]
57 [887 768]
58 [888 767]
59 [889 766]
60 [890 765]
61 [891 764]
62 [891 763]
63 [892 762]
64 [893 762]
65 [893 761]
66 [894 760]
67 [895 759]
68 [895 758]
69 [896 757]
70 [897 756]
71 [898 755]
72 [899 755]
73 [899 754]
74 [900 753]
75 [901 752]
76 [901 751]
77 [902 7

In [122]:
for i,j in enumerate(para2line(frags_para[0][0][2])):
    print(i,j)

0 [  0 532]
1 [  1 532]
2 [  2 532]
3 [  3 532]
4 [  4 532]
5 [  5 533]
6 [  6 533]
7 [  7 533]
8 [  8 533]
9 [  9 534]
10 [ 10 534]
11 [ 11 534]
12 [ 12 534]
13 [ 13 535]
14 [ 14 535]
15 [ 15 535]
16 [ 16 535]
17 [ 17 535]
18 [ 18 536]
19 [ 19 536]
20 [ 20 536]
21 [ 21 536]
22 [ 22 537]
23 [ 23 537]
24 [ 24 537]
25 [ 25 537]
26 [ 26 538]
27 [ 27 538]
28 [ 28 538]
29 [ 29 538]
30 [ 30 538]
31 [ 31 539]
32 [ 32 539]
33 [ 33 539]
34 [ 34 539]
35 [ 35 540]
36 [ 36 540]
37 [ 37 540]
38 [ 38 540]
39 [ 39 541]
40 [ 40 541]
41 [ 41 541]
42 [ 42 541]
43 [ 43 541]
44 [ 44 542]
45 [ 45 542]
46 [ 46 542]
47 [ 47 542]
48 [ 48 543]
49 [ 49 543]
50 [ 50 543]
51 [ 51 543]
52 [ 52 544]
53 [ 53 544]
54 [ 54 544]
55 [ 55 544]
56 [ 56 544]
57 [ 57 545]
58 [ 58 545]
59 [ 59 545]
60 [ 60 545]
61 [ 61 546]
62 [ 62 546]
63 [ 63 546]
64 [ 64 546]
65 [ 65 547]
66 [ 66 547]
67 [ 67 547]
68 [ 68 547]
69 [ 69 547]
70 [ 70 548]
71 [ 71 548]
72 [ 72 548]
73 [ 73 548]
74 [ 74 549]
75 [ 75 549]
76 [ 76 549]
77 [ 77 5

835 [835 724]
836 [836 724]
837 [837 725]
838 [838 725]
839 [839 725]
840 [840 725]
841 [841 726]
842 [842 726]
843 [843 726]
844 [844 726]
845 [845 727]
846 [846 727]
847 [847 727]
848 [848 727]
849 [849 727]
850 [850 728]
851 [851 728]
852 [852 728]
853 [853 728]
854 [854 729]
855 [855 729]
856 [856 729]
857 [857 729]
858 [858 730]
859 [859 730]
860 [860 730]
861 [861 730]
862 [862 730]
863 [863 731]
864 [864 731]
865 [865 731]
866 [866 731]
867 [867 732]
868 [868 732]
869 [869 732]
870 [870 732]
871 [871 733]
872 [872 733]
873 [873 733]
874 [874 733]
875 [875 733]
876 [876 734]
877 [877 734]
878 [878 734]
879 [879 734]
880 [880 735]
881 [881 735]
882 [882 735]
883 [883 735]
884 [884 736]
885 [885 736]
886 [886 736]
887 [887 736]
888 [888 736]
889 [889 737]
890 [890 737]
891 [891 737]
892 [892 737]
893 [893 738]
894 [894 738]
895 [895 738]
896 [896 738]
897 [897 739]
898 [898 739]
899 [899 739]
900 [900 739]
901 [901 739]
902 [902 740]
903 [903 740]
904 [904 740]
905 [905 740]
906 [9

1719 [1719  928]
1720 [1720  928]
1721 [1721  929]
1722 [1722  929]
1723 [1723  929]
1724 [1724  929]
1725 [1725  930]
1726 [1726  930]
1727 [1727  930]
1728 [1728  930]
1729 [1729  930]
1730 [1730  931]
1731 [1731  931]
1732 [1732  931]
1733 [1733  931]
1734 [1734  932]
1735 [1735  932]
1736 [1736  932]
1737 [1737  932]
1738 [1738  933]
1739 [1739  933]
1740 [1740  933]
1741 [1741  933]
1742 [1742  933]
1743 [1743  934]
1744 [1744  934]
1745 [1745  934]
1746 [1746  934]
1747 [1747  935]
1748 [1748  935]
1749 [1749  935]
1750 [1750  935]
1751 [1751  936]
1752 [1752  936]
1753 [1753  936]
1754 [1754  936]
1755 [1755  936]
1756 [1756  937]
1757 [1757  937]
1758 [1758  937]
1759 [1759  937]
1760 [1760  938]
1761 [1761  938]
1762 [1762  938]
1763 [1763  938]
1764 [1764  939]
1765 [1765  939]
1766 [1766  939]
1767 [1767  939]
1768 [1768  939]
1769 [1769  940]
1770 [1770  940]
1771 [1771  940]
1772 [1772  940]
1773 [1773  941]
1774 [1774  941]
1775 [1775  941]
1776 [1776  941]
1777 [1777  94

In [132]:
pt1 = cam_list[2].frag_list[0][372][86]

In [133]:
S = para2cood_S(frags_para[0][0][2])

In [134]:
F = para2cood_F(frags_para[0][0][2])

In [135]:
v1 = F - S
v2 = pt1 - S
v1 = v1/(v1[0,0]**2 + v1[0,1]**2)**(1/2)
v2 = v2/(v2[0,0]**2 + v2[0,1]**2)**(1/2)

In [136]:
np.cross(v1,v2)

array([-0.00076556])

In [ ]:
frags_para